# Example notebook - How to calculate climate metrics for aviation

In [1]:
# --- Import libraries ---
import numpy as np
from aerocm.metrics.aviation_climate_metrics_calculation import AviationClimateMetricsCalculation
from aerocm.utils.functions import emission_profile_function

In [2]:
# --- Set parameters ---

## Climate model
climate_model = "FaIR" # Choose a climate model among 'IPCC', 'GWP*', 'LWE' and 'FaIR'
start_year = 1940 # Choose a start year for the climate simulation
### The end_year parameter will be automatically calculated using time horizon (Option 1)
### or the length of the species inventory, with a check for time horizon (Option 2)

## Metrics
time_horizon = [20, 50, 100] # Choose an integer or a list of integers

## Species
species_profile = 'pulse' # Choose between 'pulse', 'step', 'combined', 'scenario'

## Option 1 ('pulse', 'step', 'combined'): standard profiles
### This option will directly generate standard emission profiles (pulse/step)
profile_start_year = 2020 # Choose the start year for the pulse/step (for FaIR: profile_start_year>start_year)
species_list = ["Contrails", "Soot"] # Choose the species to study among the ones available in the climate model

## Aditional settings
### You can use species_settings or model_settings as in the case of the AviationClimateSimulation class

In [3]:
# --- Run simulation in the case of Option 1 ---
results = AviationClimateMetricsCalculation(
    climate_model,
    start_year,
    time_horizon,
    species_profile,
    profile_start_year,
    species_list   
).run(
    include_absolute_metrics=False,  # <-- whether to include absolute metrics (default is False)
    return_df=True  # <-- whether to return resulsts as a dataframe (default is a nested dict)
)

In [4]:
# display results
results

,time_horizon,species,gwp_rf,gwp_erf,egwp_rf,egwp_erf,gtp,igtp,ratr
0,20,Contrails,87.317157,36.673206,36.673206,36.673206,10.693590,48.347866,48.347866
1,20,Soot,3940.518599,3940.518599,3940.518599,3940.518599,1038.841521,5110.737201,5110.737201
2,50,Contrails,41.451008,17.409423,17.409423,17.409423,2.678263,21.323234,21.323234
3,50,Soot,1871.510333,1871.510333,1871.510333,1871.510333,193.765518,2193.817748,2193.817748
4,100,Contrails,23.155547,9.725330,9.725330,9.725330,1.573622,11.704448,11.704448
5,100,Soot,1048.526748,1048.526748,1048.526748,1048.526748,106.366923,1171.040073,1171.040073


In [5]:
# More readable as a pivot table
results.pivot_table(index=["species", "time_horizon"])

egwp_erf      egwp_rf          gtp      gwp_erf  \
species   time_horizon                                                       
Contrails 20              36.673206    36.673206    10.693590    36.673206   
          50              17.409423    17.409423     2.678263    17.409423   
          100              9.725330     9.725330     1.573622     9.725330   
Soot      20            3940.518599  3940.518599  1038.841521  3940.518599   
          50            1871.510333  1871.510333   193.765518  1871.510333   
          100           1048.526748  1048.526748   106.366923  1048.526748   

                             gwp_rf         igtp         ratr  
species   time_horizon                                         
Contrails 20              87.317157    48.347866    48.347866  
          50              41.451008    21.323234    21.323234  
          100             23.155547    11.704448    11.704448  
Soot      20            3940.518599  5110.737201  5110.737201  
          50            1871.510333  2193.817748  2193.817748  
          100           1048.526748  1171.040073  1171.040073

In [6]:
# Select specific time horizon and/or specie and metrics
results.loc[
    (results["time_horizon"] == 50) & (results["species"] == "Contrails"),  # <- rows to select
    #["gwp_rf", "gtp", "agwp_rf"]  # <- columns to select
]

,time_horizon,species,gwp_rf,gwp_erf,egwp_rf,egwp_erf,gtp,igtp,ratr
2,50,Contrails,41.451008,17.409423,17.409423,17.409423,2.678263,21.323234,21.323234


In [7]:
# --- Run simulation in the case of Option 2 ---

## Species
species_profile_scenario = 'scenario' # Choose between 'pulse', 'step', 'combined', 'scenario'

## Option 2 ('scenario'): species inventory
### This option will directly generate the emission profiles of your choice

### Intermediate parameters for generating the inventory
if type(time_horizon) == int:
    time_horizon_max = time_horizon
else:
    time_horizon_max = max(time_horizon)
profile_start_year = 2020

### Inventory
species_inventory = {
    "CO2": emission_profile_function(start_year,
                                            profile_start_year,
                                            time_horizon_max,
                                            profile="pulse",
                                            unit_value=10**10
                                            ), # in kg
    "Contrails": emission_profile_function(start_year,
                                            profile_start_year,
                                            time_horizon_max,
                                            profile="pulse",
                                            unit_value=10**10
                                            ), # in km
    "Soot": emission_profile_function(start_year,
                                            profile_start_year,
                                            time_horizon_max,
                                            profile="pulse",
                                            unit_value=10**14
                                            ),  # in kg
}

results_scenario = AviationClimateMetricsCalculation(
    climate_model,
    start_year,
    time_horizon,
    species_profile_scenario,
    species_inventory=species_inventory,
).run(include_absolute_metrics=True, return_df=True)

In [8]:
results_scenario

,time_horizon,species,agwp_rf,agwp_erf,aegwp_rf,aegwp_erf,agtp,iagtp,atr,gwp_rf,gwp_erf,egwp_rf,egwp_erf,gtp,igtp,ratr
0,20,Contrails,2.230000e-12,9.366000e-13,9.366000e-13,9.366000e-13,6.024035e-15,4.766615e-13,2.383307e-14,87.317157,36.673206,36.673206,36.673206,10.693590,48.347866,48.347866
1,20,Soot,1.006372e-10,1.006372e-10,1.006372e-10,1.006372e-10,5.852120e-13,5.038674e-11,2.519337e-12,3940.518599,3940.518599,3940.518599,3940.518599,1038.841521,5110.737201,5110.737201
2,20,CO2,2.553908e-14,2.553908e-14,2.553908e-14,2.553908e-14,5.633313e-16,9.858997e-15,4.929498e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,50,Contrails,2.230000e-12,9.366000e-13,9.366000e-13,9.366000e-13,1.416490e-15,5.610897e-13,1.122179e-14,41.451008,17.409423,17.409423,17.409423,2.678263,21.323234,21.323234
4,50,Soot,1.006844e-10,1.006844e-10,1.006844e-10,1.006844e-10,1.024795e-13,5.772711e-11,1.154542e-12,1871.510333,1871.510333,1871.510333,1871.510333,193.765518,2193.817748,2193.817748
5,50,CO2,5.379845e-14,5.379845e-14,5.379845e-14,5.379845e-14,5.288839e-16,2.631354e-14,5.262707e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,100,Contrails,2.230000e-12,9.366000e-13,9.366000e-13,9.366000e-13,8.045229e-16,6.102080e-13,6.102080e-15,23.155547,9.725330,9.725330,9.725330,1.573622,11.704448,11.704448
7,100,Soot,1.009786e-10,1.009786e-10,1.009786e-10,1.009786e-10,5.438068e-14,6.105183e-11,6.105183e-13,1048.526748,1048.526748,1048.526748,1048.526748,106.366923,1171.040073,1171.040073
8,100,CO2,9.630522e-14,9.630522e-14,9.630522e-14,9.630522e-14,5.112556e-16,5.213471e-14,5.213471e-16,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
